<a href="https://colab.research.google.com/github/CATS70/colab/blob/main/sam2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/facebookresearch/sam2

  Cloning https://github.com/facebookresearch/sam2 to /tmp/pip-req-build-xafk3pt0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/sam2 /tmp/pip-req-build-xafk3pt0
  Resolved https://github.com/facebookresearch/sam2 to commit 2b90b9f5ceec907a1c18123530e92e794ad901a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cache

In [ ]:
# Application de segmentation d'objets parasites avec SAM2
# Pour Google Colab Pro


# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import os
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
from sam2.build_sam2 import sam_model_registry
from sam2.predictor import SamPredictor
from IPython.display import display, HTML
import gradio as gr
from google.colab import files
import json
from datetime import datetime
import io
from PIL import Image
import base64

# Configuration pour télécharger le modèle SAM2
SAM2_CHECKPOINT = "/content/drive/MyDrive/MSPR/models/sam2_b.pt"
MODEL_TYPE = "vit_b"

# Télécharger le modèle SAM2 si nécessaire
if not os.path.exists(SAM2_CHECKPOINT):
    !wget https://dl.fbaipublicfiles.com/segment_anything_2/sam2_b.pt

# Initialiser le modèle SAM2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam = sam_model_registry[MODEL_TYPE](checkpoint=SAM2_CHECKPOINT)
sam.to(device=device)
predictor = SamPredictor(sam)

# Classe pour gérer notre catalogue d'objets parasites
class ParasiteObjectCatalog:
    def __init__(self, save_dir="catalog"):
        self.save_dir = save_dir
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        self.catalog = self._load_catalog()

    def _load_catalog(self):
        catalog_file = os.path.join(self.save_dir, "catalog.json")
        if os.path.exists(catalog_file):
            with open(catalog_file, "r") as f:
                return json.load(f)
        return {"objects": []}

    def save_catalog(self):
        catalog_file = os.path.join(self.save_dir, "catalog.json")
        with open(catalog_file, "w") as f:
            json.dump(self.catalog, f, indent=2)

    def add_object(self, image_path, mask, label, bbox):
        # Sauvegarder le masque comme image
        mask_id = f"{len(self.catalog['objects'])}"
        mask_filename = f"mask_{mask_id}.png"
        mask_path = os.path.join(self.save_dir, mask_filename)

        # Convertir le masque en image et sauvegarder
        mask_img = (mask * 255).astype(np.uint8)
        cv2.imwrite(mask_path, mask_img)

        # Extraire la portion d'image correspondant à l'objet
        x1, y1, x2, y2 = bbox
        image = cv2.imread(image_path)
        object_img = image[y1:y2, x1:x2]
        object_filename = f"object_{mask_id}.png"
        object_path = os.path.join(self.save_dir, object_filename)
        cv2.imwrite(object_path, object_img)

        # Ajouter l'information au catalogue
        obj_info = {
            "id": mask_id,
            "label": label,
            "image_source": image_path,
            "mask_file": mask_filename,
            "object_file": object_filename,
            "bbox": bbox,
            "date_added": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        self.catalog["objects"].append(obj_info)
        self.save_catalog()
        return mask_id

    def get_catalog_summary(self):
        labels = {}
        for obj in self.catalog["objects"]:
            label = obj["label"]
            if label in labels:
                labels[label] += 1
            else:
                labels[label] = 1

        return {
            "total_objects": len(self.catalog["objects"]),
            "labels": labels
        }

# Fonction pour explorer Google Drive et obtenir la structure des dossiers
def explore_drive_folders(base_path="/content/drive/MyDrive/MSPR/empreintes", filter_extensions=['.jpg', '.jpeg', '.png']):
    """
    Explore les dossiers dans Google Drive et renvoie une structure d'arborescence
    avec les dossiers et les fichiers d'images.
    """
    result = {}

    # Vérifier si le chemin existe
    if not os.path.exists(base_path):
        return {"error": f"Le chemin {base_path} n'existe pas"}

    # Explorer les dossiers
    for root, dirs, files in os.walk(base_path):
        # Ne conserver que les fichiers image
        image_files = [f for f in files if any(f.lower().endswith(ext) for ext in filter_extensions)]

        if image_files:  # Conserver seulement les dossiers avec des images
            rel_path = os.path.relpath(root, base_path)
            if rel_path == '.':
                rel_path = ''

            # Créer la structure de chemin dans le dictionnaire
            current = result
            if rel_path:
                parts = rel_path.split(os.sep)
                for i, part in enumerate(parts):
                    if part not in current:
                        current[part] = {}
                    current = current[part]

            # Ajouter les fichiers image
            current['__files__'] = [os.path.join(root, f) for f in image_files]

    return result

# Initialiser notre catalogue
catalog = ParasiteObjectCatalog(save_dir="/content/drive/MyDrive/MSPR/parasite_catalog")

# Fonction pour traiter une image avec SAM2
def process_image_with_sam2(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    predictor.set_image(image)
    return image

# Fonction pour générer des masques à partir de points
def generate_masks_from_points(image, points, point_labels):
    masks, scores, logits = predictor.predict(
        point_coords=np.array(points),
        point_labels=np.array(point_labels),
        multimask_output=True,
    )
    return masks, scores

# Interface Gradio pour l'application
def create_segmentation_app():
    # Variables globales pour stocker l'état
    current_image_path = None
    current_image = None
    current_masks = None
    current_scores = None
    selected_mask_idx = 0
    drive_folder_structure = None
    current_folder_path = "/content/drive/MyDrive"

    # Fonction pour naviguer dans les dossiers Google Drive
    def load_drive_folders():
        nonlocal drive_folder_structure, current_folder_path
        drive_folder_structure = explore_drive_folders(base_path="/content/drive/MyDrive/MSPR/empreintes")
        return "Structure de Google Drive chargée. Naviguez dans vos dossiers pour trouver vos images d'empreintes."

    # Fonction pour afficher les sous-dossiers et fichiers du dossier actuel
    def get_folder_contents(folder_path):
        nonlocal drive_folder_structure, current_folder_path

        if folder_path == "..":  # Remonter d'un niveau
            current_folder_path = os.path.dirname(current_folder_path)
            if current_folder_path == "/content/drive":
                current_folder_path = "/content/drive/MyDrive"
        else:
            current_folder_path = folder_path

        # Explorer le dossier actuel
        contents = {"folders": [], "files": []}

        for item in os.listdir(current_folder_path):
            item_path = os.path.join(current_folder_path, item)
            if os.path.isdir(item_path):
                contents["folders"].append({"name": item, "path": item_path})
            elif any(item.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']):
                contents["files"].append({"name": item, "path": item_path})

        # Préparer les options pour le dropdown
        folder_options = [{"name": "..", "path": ".."}] + sorted(contents["folders"], key=lambda x: x["name"])
        file_options = sorted(contents["files"], key=lambda x: x["name"])

        return current_folder_path, folder_options, file_options

    # Fonction pour sélectionner un dossier
    def select_folder(folder_path):
        _, folder_options, file_options = get_folder_contents(folder_path)
        folder_names = [f"{folder['name']} (dossier)" for folder in folder_options]
        file_names = [f"{file['name']} (fichier)" for file in file_options]

        return gr.Dropdown.update(choices=folder_names + file_names,
                                  value=None,
                                  label=f"Contenu de {folder_path}")

    # Fonction pour sélectionner une image
    def select_item(item_name, folder_contents_dropdown):
        nonlocal current_image_path, current_image

        # Retrouver le chemin complet basé sur la sélection
        selected_item = item_name.split(" (")[0]  # Enlever le suffixe (dossier) ou (fichier)
        item_type = "dossier" if "(dossier)" in item_name else "fichier"

        if item_type == "dossier":
            # Naviguer vers ce dossier
            for folder in folder_contents_dropdown:
                if folder["name"] == selected_item:
                    return select_folder(folder["path"]), None, "Navigation vers le dossier: " + selected_item
        else:
            # Charger l'image
            for file in folder_contents_dropdown:
                if file["name"] == selected_item:
                    image_path = file["path"]
                    current_image_path = image_path
                    current_image = process_image_with_sam2(image_path)

                    # Afficher l'image
                    plt.figure(figsize=(10, 10))
                    plt.imshow(current_image)
                    plt.axis('off')
                    plt.tight_layout()

                    # Convertir le plot en image
                    buf = io.BytesIO()
                    plt.savefig(buf, format='png')
                    buf.seek(0)
                    data = base64.b64encode(buf.read()).decode('ascii')
                    plt.close()

                    return gr.Dropdown.update(), f"data:image/png;base64,{data}", f"Image chargée: {selected_item}. Cliquez sur l'image pour sélectionner les objets parasites."

        return gr.Dropdown.update(), None, "Erreur lors de la sélection de l'élément."

    def upload_image(image_file):
        nonlocal current_image_path, current_image

        # Sauvegarder l'image téléchargée
        image_path = "uploaded_image.jpg"
        with open(image_path, "wb") as f:
            f.write(image_file)

        # Traiter l'image avec SAM2
        current_image_path = image_path
        current_image = process_image_with_sam2(image_path)

        # Afficher l'image
        plt.figure(figsize=(10, 10))
        plt.imshow(current_image)
        plt.axis('off')
        plt.tight_layout()

        # Convertir le plot en image
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        data = base64.b64encode(buf.read()).decode('ascii')
        plt.close()

        return f"data:image/png;base64,{data}", "Image téléchargée avec succès. Cliquez sur l'image pour sélectionner les objets parasites."

    def segment_from_clicks(image_data, evt: gr.SelectData):
        nonlocal current_image, current_masks, current_scores, selected_mask_idx

        if current_image is None:
            return image_data, "Veuillez d'abord télécharger une image."

        # Récupérer les coordonnées du clic
        x, y = evt.index
        points = [[x, y]]
        point_labels = [1]  # 1 pour foreground

        # Générer les masques
        masks, scores = generate_masks_from_points(current_image, points, point_labels)
        current_masks = masks
        current_scores = scores
        selected_mask_idx = 0  # Sélectionner le premier masque par défaut

        # Afficher l'image avec le masque
        plt.figure(figsize=(10, 10))
        plt.imshow(current_image)

        # Superposer le masque
        show_mask(masks[selected_mask_idx], plt.gca())
        show_points(points, point_labels, plt.gca())

        plt.title(f"Score du masque: {scores[selected_mask_idx]:.3f}")
        plt.axis('off')
        plt.tight_layout()

        # Convertir le plot en image
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        data = base64.b64encode(buf.read()).decode('ascii')
        plt.close()

        # Préparer les options de masque
        mask_options = [f"Masque {i+1} (Score: {score:.3f})" for i, score in enumerate(scores)]

        return f"data:image/png;base64,{data}", f"Objet segmenté! Choisissez un masque et ajoutez-le au catalogue."

    def change_mask(mask_idx):
        nonlocal current_masks, current_scores, selected_mask_idx

        if current_masks is None:
            return None, "Veuillez d'abord segmenter un objet."

        selected_mask_idx = mask_idx

        # Afficher l'image avec le masque sélectionné
        plt.figure(figsize=(10, 10))
        plt.imshow(current_image)

        # Superposer le masque
        show_mask(current_masks[selected_mask_idx], plt.gca())

        plt.title(f"Score du masque: {current_scores[selected_mask_idx]:.3f}")
        plt.axis('off')
        plt.tight_layout()

        # Convertir le plot en image
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        data = base64.b64encode(buf.read()).decode('ascii')
        plt.close()

        return f"data:image/png;base64,{data}", f"Masque {selected_mask_idx+1} sélectionné."

    def add_to_catalog(label):
        nonlocal current_image_path, current_masks, selected_mask_idx

        if current_masks is None:
            return "Veuillez d'abord segmenter un objet."

        if not label:
            return "Veuillez entrer une étiquette pour l'objet parasite."

        # Récupérer le masque sélectionné
        mask = current_masks[selected_mask_idx]

        # Calculer la boîte englobante
        y_indices, x_indices = np.where(mask)
        x1, x2 = np.min(x_indices), np.max(x_indices)
        y1, y2 = np.min(y_indices), np.max(y_indices)
        bbox = [int(x1), int(y1), int(x2), int(y2)]

        # Ajouter au catalogue
        object_id = catalog.add_object(current_image_path, mask, label, bbox)

        # Récupérer le résumé du catalogue
        summary = catalog.get_catalog_summary()

        return f"Objet ajouté au catalogue avec ID: {object_id}\n\nRésumé du catalogue:\n- Total d'objets: {summary['total_objects']}\n- Étiquettes: {', '.join([f'{k} ({v})' for k, v in summary['labels'].items()])}"

    def export_catalog():
        # Créer un zip du catalogue
        !zip -r /content/catalog.zip /content/drive/MyDrive/MSPR/parasite_catalog

        # Télécharger le zip
        files.download('/content/catalog.zip')

        return "Catalogue exporté avec succès sous forme de fichier ZIP."

    # Fonctions d'aide pour visualiser les masques et points
    def show_mask(mask, ax):
        color = np.array([30/255, 144/255, 255/255, 0.6])
        h, w = mask.shape[-2:]
        mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
        ax.imshow(mask_image)

    def show_points(coords, labels, ax, marker_size=375):
        pos_points = coords[labels==1]
        neg_points = coords[labels==0]
        ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
        ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

    # Création de l'interface Gradio
    with gr.Blocks() as app:
        gr.Markdown("# Application de segmentation d'objets parasites avec SAM2")
gr.Markdown("#### Utilise le modèle Meta Segment Anything 2 pour détecter et cataloguer les objets parasites dans les images d'empreintes")
        gr.Markdown("Cette application vous permet de sélectionner des objets parasites dans des images d'empreintes et de créer un catalogue pour entraîner un modèle de détection.")

        with gr.Row():
            with gr.Column(scale=1):
                # Panneau de navigation dans Google Drive
                load_drive_btn = gr.Button("Charger les dossiers Google Drive")
                current_path_display = gr.Textbox(label="Chemin actuel", value="/content/drive/MyDrive/MSPR/empreintes")
                folder_browser = gr.Dropdown(label="Contenu du dossier", choices=[], interactive=True)

                # Alternative: téléchargement direct
                gr.Markdown("### Ou téléchargez directement une image:")
                upload_btn = gr.File(label="Télécharger une image")

            with gr.Column(scale=2):
                # Affichage et manipulation de l'image
                image_display = gr.Image(label="Image", interactive=True)
                status = gr.Textbox(label="Statut", value="Sélectionnez une image pour commencer.")

                with gr.Row():
                    mask_selector = gr.Slider(minimum=0, maximum=2, step=1, value=0, label="Sélectionner un masque", interactive=True)

                with gr.Row():
                    label_input = gr.Textbox(label="Étiquette de l'objet parasite")
                    add_btn = gr.Button("Ajouter au catalogue")

                catalog_status = gr.Textbox(label="Statut du catalogue", value="Aucun objet dans le catalogue.", lines=5)
                export_btn = gr.Button("Exporter le catalogue")

        # Variables pour stocker temporairement les données du navigateur de fichiers
        folder_contents = gr.State([])

        # Événements
        load_drive_btn.click(load_drive_folders, inputs=[], outputs=[status])
        load_drive_btn.click(lambda: get_folder_contents("/content/drive/MyDrive/MSPR/empreintes"),
                            inputs=[],
                            outputs=[current_path_display, folder_contents, folder_contents])
        load_drive_btn.click(lambda x: [f"{folder['name']} (dossier)" for folder in x] + [f"{file['name']} (fichier)" for file in x],
                            inputs=[folder_contents],
                            outputs=[folder_browser])

        folder_browser.change(select_item,
                             inputs=[folder_browser, folder_contents],
                             outputs=[folder_browser, image_display, status])

        upload_btn.upload(upload_image, inputs=[upload_btn], outputs=[image_display, status])
        image_display.select(segment_from_clicks, inputs=[image_display], outputs=[image_display, status])
        mask_selector.change(change_mask, inputs=[mask_selector], outputs=[image_display, status])
        add_btn.click(add_to_catalog, inputs=[label_input], outputs=[catalog_status])
        export_btn.click(export_catalog, inputs=[], outputs=[catalog_status])

    return app

# Lancer l'application
app = create_segmentation_app()
app.launch(debug=True)

# Instructions d'utilisation
print("""
Instructions d'utilisation:
1. Autorisez l'accès à votre Google Drive lorsque demandé
2. Cliquez sur 'Charger les dossiers Google Drive' pour accéder à vos images
3. Naviguez dans la structure de vos dossiers par animal et sélectionnez une image d'empreinte
4. Cliquez sur un objet parasite dans l'image pour le segmenter avec SAM
5. Utilisez le curseur pour sélectionner le meilleur masque parmi les options
6. Entrez une étiquette pour l'objet parasite (ex: 'poussière', 'cheveu', etc.)
7. Cliquez sur 'Ajouter au catalogue' pour sauvegarder l'objet
8. Répétez pour tous les objets parasites dans l'image
9. Utilisez 'Exporter le catalogue' pour télécharger votre catalogue complet

Note: Le catalogue sera enregistré dans votre Google Drive à l'emplacement /MyDrive/parasite_catalog/
Ce catalogue pourra être utilisé ultérieurement pour entraîner votre propre modèle de détection.
""")